In [1]:
!pip install google-generativeai

In [2]:
!pip install langchain


In [3]:
!pip install python-dotenv

In [4]:
!pip install tiktoken

In [5]:
!pip install faiss-cpu

In [6]:
!pip install protobuf

In [7]:
# !pip install google-generativeai

In [8]:
!pip install InstructorEmbedding

In [9]:
!pip install -U sentence-transformers

In [10]:
from langchain.llms import GooglePalm

In [11]:
llm = GooglePalm(google_api_key='',temperature=0.8)

In [12]:
llm("Write a love letter to my Wife named sahasra")

"Dear Sahasra,\n\nI can't believe it's been 5 years since we got married. It feels like just yesterday we were standing at the altar, promising to love each other forever. And I still can't believe I'm so lucky to have you as my wife.\n\nYou are the most amazing woman I know. You're kind, compassionate, intelligent, and beautiful. You make me laugh every day, and you always know how to make me feel better when I'm down.\n\nI love the way you look at me, like I'm the only man in the world. I love the way you touch me, like you can't get enough of me. And I love the way you make me feel, like I can do anything.\n\nYou are my best friend, my lover, and the mother of our children. You are the love of my life.\n\nI promise to love you forever,\nYour Husband"

In [13]:
from langchain.document_loaders.csv_loader import CSVLoader
loader = CSVLoader('codebasics_faqs.csv',source_column='prompt',encoding='cp1252')
data = loader.load()

In [14]:
data[0]

Document(page_content='prompt: I have never done programming in my life. Can I take this bootcamp?\nresponse: Yes, this is the perfect bootcamp for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in your current job or business using data.', metadata={'source': 'I have never done programming in my life. Can I take this bootcamp?', 'row': 0})

In [15]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

In [18]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

# Initialize instructor embeddings using the Hugging Face model
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large")

e = instructor_embeddings.embed_query("What is your refund policy?")

load INSTRUCTOR_Transformer
max_seq_length  512


In [21]:
e[:10]

[-0.0438980758190155,
 0.007685543969273567,
 -0.009231905452907085,
 0.024496249854564667,
 0.033592287451028824,
 0.06253517419099808,
 0.010531392879784107,
 0.021612413227558136,
 -0.02216225489974022,
 0.029747502878308296]

In [23]:
from langchain.vectorstores import FAISS
vectordb = FAISS.from_documents(documents=data,embedding= instructor_embeddings)

In [26]:
retriever = vectordb.as_retriever()
rdocs = retriever.get_relevant_documents('how about job placements ?')

In [27]:
rdocs

[Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.', metadata={'source': 'Do you provide any job assistance?', 'row': 11}),
 Document(page_content='prompt: Will this course guarantee me a job?\nresponse: We created a much lighter version of this course on YouTube available for free (click this link) and many people gave us feedback that they were able to fetch jobs (see testimonials). Now this paid course is at least 5x better than the YouTube course which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.', metadata={'source': 'Will this 

In [28]:
from langchain.chains import RetrievalQA
chain = RetrievalQA.from_chain_type(llm = llm,chain_type = 'stuff',retriever = retriever)
chain("Do you guys provide intenship and also do you guys provide EMI?")

{'query': 'Do you guys provide intenship and also do you guys provide EMI?',
 'result': 'Yes, we provide virtual internship and no EMI options.'}

In [29]:
chain("do you guys provide EMI?")

{'query': 'do you guys provide EMI?', 'result': 'No'}

In [30]:
from langchain.prompts import PromptTemplate
prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""
prompt = PromptTemplate(template = prompt_template,input_variables=['context','question'])

In [31]:
chain = RetrievalQA.from_chain_type(llm = llm,chain_type = 'stuff',retriever = retriever,chain_type_kwargs={'prompt':prompt})

In [32]:
chain('do u have plan of launching blockchain course ?')

{'query': 'do u have plan of launching blockchain course ?',
 'result': '\nresponse: We are planning to launch our next course on Blockchain & Cryptocurrencies. Stay tuned for upcoming updates on that.'}